<a name="top"></a><img src="imgs/SpinalHDL.png" alt="SpinalHDL based on Scala" style="width:320px;" />

运行SpinalHDL代码之前，务必加载SpinalHDL的库 
（**注**：*第一次运行时需要下载库，可能会有点慢，请耐心等待*）

In [ ]:
val path = System.getProperty("user.dir") + "/source/load-spinal.sc"
interp.load.module(ammonite.ops.Path(java.nio.file.FileSystems.getDefault().getPath(path)))

In [ ]:
case class RGB(channelWidth: Int) extends Bundle{
  val red   = UInt(channelWidth bit)
  val green = UInt(channelWidth bit)
  val blue  = UInt(channelWidth bit)
  def isBlack : Bool = red === 0 && green === 0 && blue === 0
} 

class MyRGB extends Component{
  val source = slave  Stream(RGB(8))
  val sink   = master Stream(RGB(8))
  sink <-< source.s2mPipe
} 

showRtl(new MyRGB)class T3 extends Component{
 val a = in(UInt(8 bits))
 val b = out(UInt(8 bits))
 b := a + 133
}

In [ ]:
object MySpinalConfig extends SpinalConfig(defaultConfigForClockDomains = ClockDomainConfig(resetKind = SYNC))

class Myflow1 extends Component{
    val source = slave  Flow(RGB(8))
    val sink   = master Flow(RGB(8))
    sink <-< source 
}

showVerilog(new Myflow1, "Myflow1")